# PQCombNet


In this tutorial, we will guide you through the usage of `PQCombNet`, a tool for training and optimizing quantum circuits for tasks such as reversing quantum processes. The `PQCombNet` class allows for flexible control over the quantum circuit, training mode, and dataset, while providing an automated process for optimizing circuit fidelity.

**Table of contents**

- [Introduction to Quantum Comb Structure](#introduction)
- [Initializing PQCombNet](#initializing)
- [Updating Quantum Circuit Parameters](#updating)
- [Training the Circuit](#training)
- [Extracting the Highest Fidelity Result](#extracting)
- [Conclusion](#conclusion)


## 1. Introduction to Quantum Comb Structure <a id="introduction"></a>

A **quantum comb** is a framework used to transform quantum processes, providing a flexible way to manipulate unitary operations. `PQCombNet` leverages **Parameterized Quantum Circuits (PQC)** to design and optimize circuits capable of performing various transformations on quantum unitary operations.

For example, `PQCombNet` can be used to train circuits that reverse unknown unitary operations, but its application extends to more general tasks such as complex conjugation and other customized quantum process transformations. By adjusting the parameters of the PQC, the network can be trained to implement different types of quantum transformations with high fidelity.

For more detailed theoretical insights and additional examples of quantum transformations, please refer to the original [paper](https://arxiv.org/abs/2403.03761).


## 2. Initializing PQCombNet <a id="initializing"></a>

To initialize the `PQCombNet` class, you need to specify several parameters, which define the behavior and structure of the quantum circuit. Below is an explanation of the key parameters:

### Key Parameters

- **`target_function`**: A callable function that defines the target quantum operation, typically used to optimize the circuit for a specific task. This function should support batch computation.

- **`num_slots`**: The number of slots.

- **`num_aux_qubits`**: The number of ancilla qubits. These qubits assist in performing the quantum operations and transformations.

- **`num_qubits_U`**: The number of qubits of unitary to be queried. The default is 1.

- **`train_unitary_info`**: Specifies the training dataset. It can either be an integer, which defines the size of an automatically generated dataset, or a custom tensor containing predefined unitary matrices for training. The default value is 2000.

- **`test_unitary_info`**: Similar to `train_unitary_info`, but for the testing dataset. The default value is 10000.

- **`train_mode`**: Defines the training mode for the quantum circuit. It can be either `"pqc"` (which is memory-efficient but slower) or `"choi"` (which is faster but uses more memory). By default, it is set to `"choi"`. If memory issues arise during training, the mode will automatically switch to `"pqc"` to ensure training continues.

- **`LR`**: The initial learning rate used for training the quantum circuit. The default value is 0.1.

- **`NUM_ITR`**: The max number of iterations to run during training. This determines how many times the model will process the training data and update the parameters. The default value is 1000.

- **`name_task`**: The name of the training task. If not provided, it will default to `"pqcomb_search_{target_function.__name__}"`, where `{target_function.__name__}` is the name of the target function. This allows for easy identification and tracking of different training tasks.

- **`seed`**: Used to set a seed for reproducibility. If specified, this ensures that the same results can be obtained when the training is repeated with the same data and parameters. The default is `None`, meaning no seed is set.

- **`is_save_data`**: A boolean value that indicates whether the training circuits should be saved. If set to `True`, the data will be saved in a directory that includes circuit lists describing combs. The default is `False`.

- **`is_auto_stop`**: A boolean value that, when set to `True`, stops the training automatically if the learning rate `LR` decreases to $10^{-3}$ of its initial value. This helps prevent unnecessary additional training once the learning rate becomes too small. The default is `True`.

- **`is_ctrl_U`**: A boolean value specifying whether to apply controlled unitary operations. If `True`, a control qubit will be added to the last ancilla qubit system, and alternating $\text{ctrl-}U$ and $\text{ctrl-}U^\dagger$ operations will be applied. If `False`, the circuit will apply $U$ without control qubits. The default is `False`.


### Example Initialization

First of all, import the necessary libraries and then there are two options for initializing the `PQCombNet` class.

In [1]:
from quairkit.application import PQCombNet

Option 1: Initialize `PQCombNet` with only the required parameters (no default values)

In [2]:
from quairkit.qinfo import dagger  # This tutorial uses the dagger function as example

num_slots = 2
num_aux_qubits = 1

net = PQCombNet(
    target_function=dagger,  # Callable function, no default value
    num_slots=num_slots,  # Number of slots, no default value
    num_aux_qubits=num_aux_qubits,  # Number of ancilla qubits, no default value
)

TypeError: PQCombNet.__init__() got an unexpected keyword argument 'num_aux_qubits'

Option 2: Initialize `PQCombNet` with all parameters

In [ ]:
num_qubits_U = 1

net = PQCombNet(
    target_function=dagger,  # Callable function, no default value
    num_slots=num_slots,  # Number of slots, no default value
    num_aux_qubits=num_aux_qubits,  # Number of ancilla qubits, no default value
    num_qubits_U=num_qubits_U,  # Default: 1 (number of qubits for unitary operations)
    train_unitary_info=2000,  # Default: 2000 (can be an integer or torch.Tensor for training data)
    test_unitary_info=10000,  # Default: 10000 (can be an integer or torch.Tensor for test data)
    train_mode="choi",  # Default: "choi" (alternatively "pqc", which is memory efficient)
    LR=0.5,  # Default: 0.1 (learning rate for optimization)
    NUM_ITR=10000,  # Default: 1000 (number of iterations for training)
    name_task="search_dag",  # Default: None (auto-generated as "pqcomb_search_{target_function.__name__}")
    seed=42,  # Default: None (random seed for reproducibility)
    is_save_data=True,  # Default: False (whether to save training data)
    is_auto_stop=True,  # Default: True (automatically stop training if learning rate decreases too much)
)

This initializes a `PQCombNet` object with the specified parameters, ready for training and optimization.


## 3. Updating Quantum Circuit Parameters <a id="updating"></a>

The `update_V_circuit` method allows you to update a specific `V` circuit at a given index within the `V_circuit_list`. It accepts various types of input, such as a `ParamOracle`, `torch.Tensor`, a tuple of a tensor and qubit indices, or an entire `Circuit` object. If no specific gate is provided for a given index, the circuit will apply universal gates by default.


### Parameters

- **`index`**:  
  An integer specifying the index of the `V` circuit in the list that you want to update.

- **`new_V`**:  
  The new circuit to replace the existing one. This can be one of the following:
  - `Circuit`: A full parameterized quantum circuit that replaces the existing one, provided it has the same number of qubits (parameterized, where the parameters will be adjusted during training).
  - `ParamOracle`: A parameterized oracle for the circuit (parameterized).
  - `torch.Tensor`: A tensor representing a fixed unitary matrix to apply to the circuit (non-parameterized, remains unchanged during training).
  - `Tuple[torch.Tensor, List[int]]`: A tuple consisting of a tensor (the fixed unitary matrix) and a list of qubit indices to which the operation will apply (non-parameterized).

### Example Usage


Prepare the new_V in several ways.

In [ ]:
from quairkit.circuit import *
from quairkit.database import *
from quairkit.operator import *

# Number of qubits in the circuit
num_qubits_cir = num_aux_qubits + num_qubits_U

# Circuit
V0 = Circuit(num_qubits_cir)
V0.rx()


# ParamOracle
def V_generator(param):
    return rx(param[0]) @ u3(param[1:4]) @ h()

V1 = ParamOracle(V_generator, num_acted_param=4)

# torch.Tensor as a gate working on all the systems
V2 = random_unitary(num_qubits_cir)  # Work on all the systems

# torch.Tensor as a gate working on parts of the systems
V3 = random_unitary(num_qubits_cir - 1)

Update the V circuit in place at a specified `index`. The `index` must satisfy the condition $0 \leq \text{index} < num_slots+1$, where `num_slots` is the number of slots in the quantum comb. This ensures that the specified index is within the valid range of the circuit list.

In [ ]:
net.update_V_circuit(index=1, new_V=V0)
# net.update_V_circuit(index=1, new_V=V1)
# net.update_V_circuit(index=1, new_V=V2)
# net.update_V_circuit(index=1, new_V=(V3, list(range(1, num_qubits_cir))))

## 4. Training the Circuit <a id="training"></a>

The core of `PQCombNet` is its ability to train quantum circuits. The `train()` method runs the training process, optimizing the circuit parameters based on the provided dataset. 

During training, log information is not only printed to the console but is also saved to the file system. Logs are stored in the `data_directory_name` (`{name_task}_data`) folder under the filename `{name_task}_train_log.csv`, allowing for easy access and review of the training process details later.

### Example Usage

In [ ]:
net.train()

[search_dag | choi | 42 | 0	0.0108s] num_qubits_U: 1, num_slots: 2, num_aux_qubits: 1, LR: 5.00e-01, Loss: 0.73879212, Fid: 0.26230755
[search_dag | choi | 42 | 40	0.0100s] num_qubits_U: 1, num_slots: 2, num_aux_qubits: 1, LR: 5.00e-01, Loss: 0.49928927, Fid: 0.49991816
[search_dag | choi | 42 | 80	0.0110s] num_qubits_U: 1, num_slots: 2, num_aux_qubits: 1, LR: 5.00e-01, Loss: 0.49813539, Fid: 0.50312698
[search_dag | choi | 42 | 120	0.0110s] num_qubits_U: 1, num_slots: 2, num_aux_qubits: 1, LR: 5.00e-05, Loss: 0.49811590, Fid: 0.50300652
[search_dag | choi | 42] Finished training with Fidelity: 0.50300652


## 5. Extracting the Highest Fidelity Result <a id="extracting"></a>

Once the circuit is trained, you can use `extract_highest_fidelity()` to find and save the best-performing circuit based on fidelity.

The function saves several output files:

- A CSV file containing a pivot table showing the highest fidelity values across different configurations of `num_slots` and `num_aux_qubits`.
- CSV files for different `num_qubits_U` values, storing fidelity results in corresponding directories.

### Example Usage


In [ ]:
net.extract_highest_fidelity()

Saved table for num_qubits_U = 1 to search_dag_data\fidelity_tables\fidelity_table_num_qubits_U_1.csv


### File Saving

- **Training Data**: If `is_save_data=True`, the training data will be saved in the directory `data_directory_name/V_circuit_lists/`. Each saved file includes details like the mode used, number of qubits, and fidelity.
- **Logs**: Training logs are saved as CSV files named `{name_task}_train_log.csv` in the specified data directory.
- **Fidelity Results**: After extracting the highest fidelity, results for each `num_qubits_U` are saved as CSV files with names like `fidelity_table_num_qubits_U_{num_qubits_U}.csv`.


## Conclusion <a id="conclusion"></a>

Throughout this tutorial, we have explored the capabilities and functionalities of the `PQCombNet` class for optimizing quantum circuits. We began by detailing how to initialize the class with various parameters tailored to specific quantum tasks. This setup allows for the flexible adaptation of `PQCombNet` to different quantum operations and transformations.

Following initialization, we discussed how to update individual circuits within the quantum comb using the `update_V_circuit` method. This method provides the flexibility needed to modify quantum circuits dynamically based on evolving requirements during training.

The core of `PQCombNet` is its training process, facilitated by the `train()` method, which optimizes quantum circuits based on provided datasets. Notably, the `extract_highest_fidelity` method plays a crucial role at the end of training by identifying and retrieving the circuit configuration that achieves the highest fidelity, ensuring the best possible performance.

Logs and detailed training outputs are not only visible in real-time but are also systematically saved to CSV files, making the training process transparent and traceable. This meticulous documentation supports thorough analysis and fine-tuning of quantum circuits.

The `PQCombNet` offers a robust framework for researchers and developers working in quantum computing, particularly those focused on circuit design and optimization.

Future explorations can leverage the extensibility of `PQCombNet` to experiment with more complex quantum operations. The potential for further enhancements and its application in solving intricate quantum problems presents an exciting avenue for ongoing research and development.


---

## References
[1] Mo, Yin, et al. "Parameterized quantum comb and simpler circuits for reversing unknown qubit-unitary operations." arXiv preprint arXiv:2403.03761 (2024).


In [ ]:
import quairkit as qkit

qkit.print_info()


---------VERSION---------
quairkit: 0.1.0
torch: 2.3.1+cpu
numpy: 1.26.4
scipy: 1.14.0
matplotlib: 3.9.0
---------SYSTEM---------
Python version: 3.10.14
OS: Windows
OS version: 10.0.22621
---------DEVICE---------
CPU: AMD64 Family 25 Model 68 Stepping 1, AuthenticAMD
